### Ingest races.csv file

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read the CSV file using the spark dataframe reader API

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import to_timestamp, concat, col, lit

races_schema = StructType(
    fields=[
        StructField("raceId", IntegerType(), False),
        StructField("year", IntegerType(), True),
        StructField("round", IntegerType(), True),
        StructField("circuitId", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("date", DateType(), True),
        StructField("time", StringType(), True),
        StructField("url", StringType(), True),
    ]
)

# Read from Landing Zone
races_df = (
    spark.read.option("header", True)
    .schema(races_schema)
    .csv(f"{landing_folder_path}/{v_file_date}/races.csv")
)

##### Step 2 - Add ingestion meta data

In [ ]:
races_with_metadata_df = (
    add_ingestion_date(races_df)
    .withColumn("data_source", lit(v_data_source))
    .withColumn("file_date", lit(v_file_date))
)

##### Step 3 - Write the output to bronze container

In [ ]:
races_with_metadata_df.write.mode("overwrite").partitionBy("year").format(
    "delta"
).saveAsTable("f1_bronze.races")